In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv('data.csv')

In [3]:
newData = data.loc[data['case_status'].astype(str).str.lower() == "certified"]

#####  Assuming a employee works for 40 hours per week and each month has 4 weeks. So to convert hourly salary to yearly, I multiple hourly salary * 4 * 40 * 12.
##### Current approach for Nan in wage_unit, if wage_offer is less than 100, I am assuming, it will be an hourly salary. So assigning and the wages to show yearly salary

In [4]:
for idi, row in newData.iterrows():
    if str(row['wage_unit']).lower() == 'hour':
        newData.loc[idi, 'wage_offer'] = newData.loc[idi, 'wage_offer'] * 40 * 4 * 12
        newData.loc[idi, 'wage_unit'] = 'Year'
        
    elif not pd.notnull(row['wage_unit']):
        if newData.loc[idi, 'wage_offer'] < 100:
            newData.loc[idi, 'wage_offer'] = newData.loc[idi, 'wage_offer'] * 40 * 4 * 12
            newData.loc[idi, 'wage_unit'] = 'Year'
        else:
            newData.loc[idi, 'wage_unit'] = 'Year'

/home/rohit/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


##### Since all the wages are in yearly form, I drop 'wage_unit' column

In [5]:
newData = newData.drop('wage_unit', axis = 1)

##### We don't need 'job_foreign_lang_req', 'case_number', 'case_received_date', 'decision_date', 'employer_yr_established'

In [6]:
newData = newData.drop(['job_foreign_lang_req', 'case_number', 'case_received_date', 'decision_date', 'employer_yr_established'], axis = 1)

In [7]:
def num_missing(x):
  return sum(x.isnull())

In [8]:
print newData.apply(num_missing, axis=0)

case_status                      0
employer_name                    0
employer_num_employees           0
job_education                    0
job_experience_num_months    16933
job_state                        0
job_level                     2653
employee_citizenship             0
wage_offer                       0
dtype: int64


##### NAN in 'job_experience_num_months'. I am grouping the 'job_level' values and for each 'job_level' value, I am taking the average of present 'job_experience_num_months' and I am assigning it to the NAN values of 'job_experience_num_months' for that 'job_level'.
##### Because of the above assumption, I am deleting the rows where 'job_level' are NAN, in most of the cases, 'job_experience_num_months' and 'job_level' are NAN.

In [9]:
newData = newData.loc[newData['job_level'].astype(str).str.lower() != 'nan']

In [10]:
print newData.apply(num_missing, axis=0)

case_status                      0
employer_name                    0
employer_num_employees           0
job_education                    0
job_experience_num_months    15430
job_state                        0
job_level                        0
employee_citizenship             0
wage_offer                       0
dtype: int64


In [11]:
def fill_job_months(df):
    filter_df1 = df.loc[df['job_experience_num_months'].astype(str).str.lower() != 'nan']
    group_job_level_df = filter_df1.groupby('job_level')['job_experience_num_months'].mean()
    for i, row in df.iterrows():
        if str(row['job_experience_num_months']).lower() == 'nan':
            if str(row['job_level']).lower() != 'nan':
                df.loc[i, 'job_experience_num_months'] = group_job_level_df[row['job_level']]
    return df

In [12]:
newData = fill_job_months(newData)

##### We don't need 'case_status' because all of them are 'Certified'

In [13]:
newData = newData.drop('case_status', axis = 1)

In [14]:
print newData.apply(num_missing, axis=0)

employer_name                0
employer_num_employees       0
job_education                0
job_experience_num_months    0
job_state                    0
job_level                    0
employee_citizenship         0
wage_offer                   0
dtype: int64


##### Now I have zero NAN values in every column. Now important features can be extracted and model can be trained.

##### Initially I am performing one hot enconding on categorical features

In [15]:
def change_categorical(data, cat):
    le = preprocessing.LabelEncoder()
    le.fit(data.values[:, cat])
    return le.transform(data.values[:, cat])

In [16]:
categories = [0, 2, 4, 6]
name = {0: 'employer_name', 2: 'job_education', 4: 'job_state', 6: 'employee_citizenship'}
for cat in categories:
    newData[name[cat]] = change_categorical(newData, cat)

In [17]:
newData

,employer_name,employer_num_employees,job_education,job_experience_num_months,job_state,job_level,employee_citizenship,wage_offer
0,1483,149.0,1,19.822651,4,1.0,119,30285.0
2,10843,5600.0,2,17.342026,52,2.0,65,125000.0
3,10843,5600.0,2,84.000000,52,3.0,29,105300.0
4,10843,5600.0,2,19.822651,52,1.0,5,114285.0
5,10843,5600.0,2,17.342026,52,2.0,29,88400.0
9,2248,48.0,4,24.000000,15,2.0,157,60258.0
14,6903,8.0,5,12.000000,34,2.0,155,48076.8
24,6335,90.0,1,60.000000,34,2.0,114,93700.0
28,3552,40.0,5,36.000000,34,3.0,29,81000.0
35,526,33010.0,1,60.000000,2,3.0,65,87500.0


In [18]:
feature_matrix_df = newData.iloc[:, :-1]
labels_df = newData.iloc[:, -1]
feature_matrix = feature_matrix_df.values
labels = labels_df.values

train_data, test_data, train_labels, test_labels = train_test_split(feature_matrix, labels, test_size=0.2, random_state=42)

##### Linear Regression

In [19]:
reg = LinearRegression().fit(train_data, train_labels)

In [20]:
pred = reg.predict(test_data)

In [21]:
mean_absolute_error(test_labels, pred)

36842.990892288515

##### Decision Tree Regressor

In [22]:
tree_regressor = DecisionTreeRegressor(random_state = 0).fit(train_data, train_labels)
pred1 = tree_regressor.predict(test_data)

In [23]:
mean_absolute_error(test_labels, pred1)

30371.633567632824

##### Random Forest Regressor

In [24]:
forest_regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)
forest_regressor.fit(train_data, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [25]:
pred2 = forest_regressor.predict(test_data)

In [26]:
mean_absolute_error(test_labels, pred2)

26649.617464589526